In [1]:
# this part is for the nondetrend plots
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import Cdo
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof

In [17]:
#   read data from observation and reanalysis data

fpreGPCP = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/GPCP_r144x72_197901-201412.nc"
)
preGPCP = fpreGPCP["precip"]
preGPCP_JJA = ca.p_time(preGPCP, 6, 8, True)

preAIR = xr.open_dataarray("/home/ys17-23/Extension/All_India_Rainfall_index/AIR_mmperday.nc")
preAIR_JJA = ca.p_time(preAIR, 6, 8, True)
preAIR_JJA = preAIR_JJA.sel(time=(preAIR_JJA.time.dt.year>=1979) & (preAIR_JJA.time.dt.year <=2014))


fhgtERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/hgt_mon_r144x72_195001-201412.nc")
hgtERA5 = fhgtERA5["z"].sel(time=fhgtERA5["time"].dt.year>=1979)

fuERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/uwind_mon_r144x72_195001-201412.nc"
)
uERA5 = fuERA5["u"].sel(time=fuERA5["time"].dt.year>=1979)

fvERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/vwind_mon_r144x72_195001-201412.nc"
)
vERA5 = fvERA5["v"].sel(time=fvERA5["time"].dt.year>=1979)

fspERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/sp_mon_r144x72_195001-201412.nc"
)
spERA5 = fspERA5["sp"].sel(time=fspERA5["time"].dt.year>=1979)

fqERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/q_mon_r144x72_195001-201412.nc")
qERA5 = fqERA5["q"].sel(time=fqERA5["time"].dt.year>=1979)

fwERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/omega_mon_r144x72_195001-201412.nc")
wERA5 = fwERA5["w"].sel(time=fwERA5["time"].dt.year>=1979)

fsstHad = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/HadISST_r144x72_1870-2020.nc")
sstHad = fsstHad["sst"].sel(time=(fsstHad["time"].dt.year>=1979) & (fsstHad["time"].dt.year<=2014))

hgtERA5_ver_JJA = ca.p_time(hgtERA5, 6, 8, True)
hgtERA5_ver_JJA = hgtERA5_ver_JJA-hgtERA5_ver_JJA.mean(dim="lon", skipna=True)
uERA5_ver_JJA = ca.p_time(uERA5, 6, 8, True)
vERA5_ver_JJA = ca.p_time(vERA5, 6, 8, True)
qERA5_ver_JJA = ca.p_time(qERA5, 6, 9, True)
spERA5_JJA = ca.p_time(spERA5, 6, 8, True)
wERA5_JJA = ca.p_time(wERA5, 6, 8, True)
sstHad_JJA = ca.p_time(sstHad, 6, 8, True)

ftERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/temp_mon_r144x72_195001-201412.nc")
tERA5 = ftERA5["t"].sel(time=ftERA5["time"].dt.year>=1979)
tERA5_ver_JJA = ca.p_time(tERA5, 6, 8, True)

divERA5_ver_JJA = ca.cal_divergence(uERA5_ver_JJA.sel(level=[200.0, 850.0]), vERA5_ver_JJA.sel(level=[200.0, 850.0]))


#   calculate the waver vapor vertical intergration
ptop = 1 * 100
g = 9.8
ERA5level = qERA5_ver_JJA.coords["level"] * 100.0
ERA5level.attrs["units"] = "Pa"
ERA5dp = geocat.comp.dpres_plevel(ERA5level, spERA5_JJA, ptop)
ERA5dpg = ERA5dp / g
ERA5dpg.attrs["units"] = "kg/m2"
# calculate the water vapor transport
uq_sl_ERA5_JJA = uERA5_ver_JJA * qERA5_ver_JJA.data * 1000.0
vq_sl_ERA5_JJA = vERA5_ver_JJA * qERA5_ver_JJA.data * 1000.0
uq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
vq_sl_ERA5_JJA.attrs["units"] = "[m/s][g/kg]"
# calculate the whole levels water vapor transport
uqERA5_ver_JJA = (uq_sl_ERA5_JJA * ERA5dpg.data).sum(dim="level") / 1e05
vqERA5_ver_JJA = (vq_sl_ERA5_JJA * ERA5dpg.data).sum(dim="level") / 1e05
uqERA5_ver_JJA.attrs["units"] = "100kg/(m*s)"
vqERA5_ver_JJA.attrs["units"] = "100kg/(m*s)"

# calculate the water vapor transportation divergence
uqvqdivERA5_ver_JJA = ca.cal_divergence(uqERA5_ver_JJA, vqERA5_ver_JJA)

In [3]:
# read the data from CMIP6 historical experiment
fprehis = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/pr_historical_r144x72_195001-201412.nc")
prehis_JJA = fprehis["pr"].sel(time=fprehis["time"].dt.year>=1979)
prehis_JJA.attrs["units"] = "mm/day"
prehis_JJA.attrs["standard_name"] = "precipitation"

fhgthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/zg_historical_r144x72_195001-201412.nc")
hgthis_ver_JJA = fhgthis_ver_JJA["zg"].sel(time=fhgthis_ver_JJA["time"].dt.year>=1979)
hgthis_ver_JJA = hgthis_ver_JJA - hgthis_ver_JJA.mean(dim="lon", skipna=True)

fuhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/ua_historical_r144x72_195001-201412.nc")
uhis_ver_JJA = fuhis_ver_JJA["ua"].sel(time=fuhis_ver_JJA["time"].dt.year>=1979)

fvhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/va_historical_r144x72_195001-201412.nc")
vhis_ver_JJA = fvhis_ver_JJA["va"].sel(time=fvhis_ver_JJA["time"].dt.year>=1979)

fwhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/wap_historical_r144x72_195001-201412.nc") 
whis_ver_JJA = fwhis_ver_JJA["wap"].sel(time=fwhis_ver_JJA["time"].dt.year>=1979)

fssthis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/tos_historical_r144x72_195001-201412.nc")
ssthis_JJA = fssthis_JJA["tos"].sel(time=fssthis_JJA["time"].dt.year>=1979)

fthis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/ta_historical_r144x72_195001-201412.nc")
this_ver_JJA = fthis_ver_JJA["ta"].sel(time=fthis_ver_JJA["time"].dt.year>=1979)

fuqhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/his_uq_dpg.nc")
uqhis_ver_JJA = fuqhis_ver_JJA["uq_dpg"].sel(time=fuqhis_ver_JJA["time"].dt.year>=1979)

fvqhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/his_vq_dpg.nc")
vqhis_ver_JJA = fvqhis_ver_JJA["vq_dpg"].sel(time=fvqhis_ver_JJA["time"].dt.year>=1979)

fuqvqdivhis_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/his_div_uqvq.nc")
uqvqdivhis_ver_JJA = fuqvqdivhis_ver_JJA["div_uqvq"].sel(time=fuqvqdivhis_ver_JJA["time"].dt.year>=1979)

divhis_ver_JJA = ca.cal_divergence(uhis_ver_JJA.sel(level=[200.0, 850.0]), vhis_ver_JJA.sel(level=[200.0, 850.0]))

In [4]:
# read the data from CMIP6 ssp585 experiment and read the last period of ssp585
fpressp585_p3 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/pr_ssp585_r144x72_201501-209912.nc")
pressp585_p3_JJA = fpressp585_p3["pr"].sel(time=fpressp585_p3.time.dt.year>=2064)
pressp585_p3_JJA.attrs["units"] = "mm/day"
pressp585_p3_JJA.attrs["standard_name"] = "precipitation"

fhgtssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/zg_ssp585_r144x72_201501-209912.nc")
hgtssp585_p3_ver_JJA = fhgtssp585_p3_ver_JJA["zg"].sel(time=fhgtssp585_p3_ver_JJA.time.dt.year>=2064)
hgtssp585_p3_ver_JJA = hgtssp585_p3_ver_JJA - hgtssp585_p3_ver_JJA.mean(dim="lon", skipna=True)


fussp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ua_ssp585_r144x72_201501-209912.nc")
ussp585_p3_ver_JJA = fussp585_p3_ver_JJA["ua"].sel(time=fussp585_p3_ver_JJA.time.dt.year>=2064)

fvssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/va_ssp585_r144x72_201501-209912.nc")
vssp585_p3_ver_JJA = fvssp585_p3_ver_JJA["va"].sel(time=fvssp585_p3_ver_JJA.time.dt.year>=2064)

fwssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/wap_ssp585_r144x72_201501-209912.nc")
wssp585_p3_ver_JJA = fwssp585_p3_ver_JJA["wap"].sel(time=fwssp585_p3_ver_JJA.time.dt.year>=2064)

fsstssp585_p3_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/tos_ssp585_r144x72_201501-209912.nc")
sstssp585_p3_JJA = fsstssp585_p3_JJA["tos"].sel(time=fsstssp585_p3_JJA.time.dt.year>=2064)

ftssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ta_ssp585_r144x72_201501-209912.nc")
tssp585_p3_ver_JJA = ftssp585_p3_ver_JJA["ta"].sel(time=ftssp585_p3_ver_JJA.time.dt.year>=2064)

divssp585_p3_ver_JJA = ca.cal_divergence(ussp585_p3_ver_JJA.sel(level=[200.0, 850.0]), vssp585_p3_ver_JJA.sel(level=[200.0, 850.0]))

fuqssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ssp585_uq_dpg.nc")
uqssp585_p3_ver_JJA = fuqssp585_p3_ver_JJA["uq_dpg"].sel(time=fuqssp585_p3_ver_JJA["time"].dt.year>=2064)

fvqssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ssp585_vq_dpg.nc")
vqssp585_p3_ver_JJA = fvqssp585_p3_ver_JJA["vq_dpg"].sel(time=fvqssp585_p3_ver_JJA["time"].dt.year>=2064)

fuqvqdivssp585_p3_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ssp585_div_uqvq.nc")
uqvqdivssp585_p3_ver_JJA = fuqvqdivssp585_p3_ver_JJA["div_uqvq"].sel(time=fuqvqdivssp585_p3_ver_JJA["time"].dt.year>=2064)

In [5]:
#   deal with the time index for CRU and GPCP data
# preCRU_JJA.coords["time"] = prehis_JJA.coords["time"]
preGPCP_JJA.coords["time"] = prehis_JJA.sel(time=prehis_JJA.time.dt.year>=1979).coords["time"]
preAIR_JJA.coords["time"] = prehis_JJA.coords["time"]

models = uhis_ver_JJA.coords["models"]
models_array = models.data

#   pick up the area data
#   calculate the vorticity in ERA5, historical and ssp585
vorERA5_ver_JJA = mpcalc.vorticity(uERA5_ver_JJA.sel(level=200.0), vERA5_ver_JJA.sel(level=200.0))
vorERA5_ver_JJA = vorERA5_ver_JJA.metpy.dequantify()

vorhis_ver_JJA = mpcalc.vorticity(uhis_ver_JJA.sel(level=200.0), vhis_ver_JJA.sel(level=200.0))
vorhis_ver_JJA = vorhis_ver_JJA.metpy.dequantify()

vorssp585_p3_ver_JJA = mpcalc.vorticity(ussp585_p3_ver_JJA.sel(level=200.0), vssp585_p3_ver_JJA.sel(level=200.0))
vorssp585_p3_ver_JJA = vorssp585_p3_ver_JJA.metpy.dequantify()

#   calculate the precipitation in India
lat = preGPCP_JJA.coords["lat"]
lon = preGPCP_JJA.coords["lon"]

India_N = 32.5
# India_N = 30.0
India_S = 7.5
India_W = 70.0
India_E = 85.0
lat_India_range = lat[(lat >= India_S) & (lat <= India_N)]
lon_India_range = lon[(lon >= India_W) & (lon <= India_E)]

# preCRU_India_JJA = ca.cal_lat_weighted_mean(preCRU_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)
preGPCP_India_JJA = ca.cal_lat_weighted_mean(preGPCP_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)
preGPCP_India_JJA = ca.standardize(preGPCP_India_JJA)
preAIR_JJA = ca.standardize(preAIR_JJA)
prehis_India_JJA = ca.cal_lat_weighted_mean(prehis_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)
prehis_India_JJA = ca.standardize(prehis_India_JJA)
pressp585_p3_India_JJA = ca.cal_lat_weighted_mean(pressp585_p3_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)
pressp585_p3_India_JJA =  ca.standardize(pressp585_p3_India_JJA)
prehis_India_JJA_sum = prehis_JJA.sel(lat=lat_India_range, lon=lon_India_range).sum(dim=["lat", "lon"], skipna=True)
pressp585_p3_India_JJA_sum = pressp585_p3_JJA.sel(lat=lat_India_range, lon=lon_India_range).sum(dim=["lat", "lon"], skipna=True)

In [6]:
# calculate the omega regress onto the precipitation
preAIR_JJA.coords["time"] = uqERA5_ver_JJA.coords["time"]
(
    IndRAIR_ERA5_uq_slope,
    IndRAIR_ERA5_uq_intercept,
    IndRAIR_ERA5_uq_rvalue,
    IndRAIR_ERA5_uq_pvalue,
    IndRAIR_ERA5_uq_hypothesis,
) = ca.dim_linregress(preAIR_JJA, uqERA5_ver_JJA)

(
    IndR_his_uq_slope,
    IndR_his_uq_intercept,
    IndR_his_uq_rvalue,
    IndR_his_uq_pvalue,
    IndR_his_uq_hypothesis,
) = ca.dim_linregress(prehis_India_JJA, uqhis_ver_JJA)

(
    IndR_ssp585_p3_uq_slope,
    IndR_ssp585_p3_uq_intercept,
    IndR_ssp585_p3_uq_rvalue,
    IndR_ssp585_p3_uq_pvalue,
    IndR_ssp585_p3_uq_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA, uqssp585_p3_ver_JJA)

(
    IndRAIR_ERA5_vq_slope,
    IndRAIR_ERA5_vq_intercept,
    IndRAIR_ERA5_vq_rvalue,
    IndRAIR_ERA5_vq_pvalue,
    IndRAIR_ERA5_vq_hypothesis,
) = ca.dim_linregress(preAIR_JJA, vqERA5_ver_JJA)

(
    IndR_his_vq_slope,
    IndR_his_vq_intercept,
    IndR_his_vq_rvalue,
    IndR_his_vq_pvalue,
    IndR_his_vq_hypothesis,
) = ca.dim_linregress(prehis_India_JJA, vqhis_ver_JJA)

(
    IndR_ssp585_p3_vq_slope,
    IndR_ssp585_p3_vq_intercept,
    IndR_ssp585_p3_vq_rvalue,
    IndR_ssp585_p3_vq_pvalue,
    IndR_ssp585_p3_vq_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA, vqssp585_p3_ver_JJA)

(
    IndRAIR_ERA5_uqvqdiv_slope,
    IndRAIR_ERA5_uqvqdiv_intercept,
    IndRAIR_ERA5_uqvqdiv_rvalue,
    IndRAIR_ERA5_uqvqdiv_pvalue,
    IndRAIR_ERA5_uqvqdiv_hypothesis,
) = ca.dim_linregress(preAIR_JJA, uqvqdivERA5_ver_JJA)

(
    IndR_his_uqvqdiv_slope,
    IndR_his_uqvqdiv_intercept,
    IndR_his_uqvqdiv_rvalue,
    IndR_his_uqvqdiv_pvalue,
    IndR_his_uqvqdiv_hypothesis,
) = ca.dim_linregress(prehis_India_JJA, uqvqdivhis_ver_JJA)

(
    IndR_ssp585_p3_uqvqdiv_slope,
    IndR_ssp585_p3_uqvqdiv_intercept,
    IndR_ssp585_p3_uqvqdiv_rvalue,
    IndR_ssp585_p3_uqvqdiv_pvalue,
    IndR_ssp585_p3_uqvqdiv_hypothesis,
) = ca.dim_linregress(pressp585_p3_India_JJA, uqvqdivssp585_p3_ver_JJA)

In [8]:
#   save the regression results

lat=IndRAIR_ERA5_uq_slope.coords["lat"]
lon=IndRAIR_ERA5_uq_slope.coords["lon"]

IndRAIR_ERA5_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndRAIR_ERA5_uq_slope.data),
        intercept=(["lat", "lon"], IndRAIR_ERA5_uq_intercept.data),
        rvalue=(["lat", "lon"], IndRAIR_ERA5_uq_rvalue.data),
        pvalue=(["lat", "lon"], IndRAIR_ERA5_uq_pvalue.data),
        hypothesis=(["lat", "lon"], IndRAIR_ERA5_uq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of ERA5 regress onto 1979-2014 AIR"),
)

IndR_his_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_uq_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_uq_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_uq_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_uq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_uq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of historical regress onto 1979-2014 IndR"),
)

IndR_ssp585_p3_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_uq_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_uq_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_uq_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_uq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_uq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of historical regress onto 2064-2099 IndR"),
)

IndRAIR_ERA5_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndRAIR_ERA5_vq_slope.data),
        intercept=(["lat", "lon"], IndRAIR_ERA5_vq_intercept.data),
        rvalue=(["lat", "lon"], IndRAIR_ERA5_vq_rvalue.data),
        pvalue=(["lat", "lon"], IndRAIR_ERA5_vq_pvalue.data),
        hypothesis=(["lat", "lon"], IndRAIR_ERA5_vq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of ERA5 regress onto 1979-2014 AIR"),
)

IndR_his_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_vq_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_vq_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_vq_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_vq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_vq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of historical regress onto 1979-2014 IndR"),
)

IndR_ssp585_p3_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_vq_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_vq_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_vq_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_vq_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_vq_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of historical regress onto 2064-2099 IndR"),
)

IndRAIR_ERA5_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_slope.data),
        intercept=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_intercept.data),
        rvalue=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_rvalue.data),
        pvalue=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_pvalue.data),
        hypothesis=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of ERA5 regress onto 1979-2014 AIR"),
)

IndR_his_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_his_uqvqdiv_slope.data),
        intercept=(["models", "lat", "lon"], IndR_his_uqvqdiv_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_his_uqvqdiv_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_his_uqvqdiv_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_his_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of historical regress onto 1979-2014 IndR"),
)

IndR_ssp585_p3_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_slope.data),
        intercept=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_intercept.data),
        rvalue=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_rvalue.data),
        pvalue=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_pvalue.data),
        hypothesis=(["models", "lat", "lon"], IndR_ssp585_p3_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        models=models.data,
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of historical regress onto 2064-2099 IndR"),
)

IndRAIR_ERA5_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_uq_regress.nc")
IndR_his_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_his_uq_regress.nc")
IndR_ssp585_p3_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_ssp585_p3_uq_regress.nc")
IndRAIR_ERA5_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_vq_regress.nc")
IndR_his_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_his_vq_regress.nc")
IndR_ssp585_p3_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_ssp585_p3_vq_regress.nc")
IndRAIR_ERA5_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_uqvqdiv_regress.nc")
IndR_his_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_his_uqvqdiv_regress.nc")
IndR_ssp585_p3_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_ssp585_p3_uqvqdiv_regress.nc")

In [ ]:
# read the water vapor transportation regression results
IndRAIR_ERA5_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_uq_regress.nc")
IndR_his_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_his_uq_regress.nc")
IndR_ssp585_p3_uq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_ssp585_p3_uq_regress.nc")
IndRAIR_ERA5_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_vq_regress.nc")
IndR_his_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_his_vq_regress.nc")
IndR_ssp585_p3_vq_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_ssp585_p3_vq_regress.nc")
IndRAIR_ERA5_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_uqvqdiv_regress.nc")
IndR_his_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_his_uqvqdiv_regress.nc")
IndR_ssp585_p3_uqvqdiv_regress = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndR_ssp585_p3_uqvqdiv_regress.nc")

IndRAIR_ERA5_uq_slope = IndRAIR_ERA5_uq_regress["slope"]
IndRAIR_ERA5_uq_rvalue = IndRAIR_ERA5_uq_regress["rvalue"]
IndRAIR_ERA5_uq_pvalue = IndRAIR_ERA5_uq_regress["pvalue"]

IndRAIR_ERA5_vq_slope = IndRAIR_ERA5_vq_regress["slope"]
IndRAIR_ERA5_vq_rvalue = IndRAIR_ERA5_vq_regress["rvalue"]
IndRAIR_ERA5_vq_pvalue = IndRAIR_ERA5_vq_regress["pvalue"]

IndRAIR_ERA5_uqvqdiv_slope = IndRAIR_ERA5_uqvqdiv_regress["slope"]
IndRAIR_ERA5_uqvqdiv_rvalue = IndRAIR_ERA5_uqvqdiv_regress["rvalue"]
IndRAIR_ERA5_uqvqdiv_pvalue = IndRAIR_ERA5_uqvqdiv_regress["pvalue"]

IndR_his_uq_slope = IndR_his_uq_regress["slope"]
IndR_his_uq_rvalue = IndR_his_uq_regress["rvalue"]
IndR_his_uq_pvalue = IndR_his_uq_regress["pvalue"]

IndR_his_vq_slope = IndR_his_vq_regress["slope"]
IndR_his_vq_rvalue = IndR_his_vq_regress["rvalue"]
IndR_his_vq_pvalue = IndR_his_vq_regress["pvalue"]

IndR_his_uqvqdiv_slope = IndR_his_uqvqdiv_regress["slope"]
IndR_his_uqvqdiv_rvalue = IndR_his_uqvqdiv_regress["rvalue"]
IndR_his_uqvqdiv_pvalue = IndR_his_uqvqdiv_regress["pvalue"]

IndR_ssp585_p3_uq_slope = IndR_ssp585_p3_uq_regress["slope"]
IndR_ssp585_p3_uq_rvalue = IndR_ssp585_p3_uq_regress["rvalue"]
IndR_ssp585_p3_uq_pvalue = IndR_ssp585_p3_uq_regress["pvalue"]

IndR_ssp585_p3_vq_slope = IndR_ssp585_p3_vq_regress["slope"]
IndR_ssp585_p3_vq_rvalue = IndR_ssp585_p3_vq_regress["rvalue"]
IndR_ssp585_p3_vq_pvalue = IndR_ssp585_p3_vq_regress["pvalue"]

IndR_ssp585_p3_uqvqdiv_slope = IndR_ssp585_p3_uqvqdiv_regress["slope"]
IndR_ssp585_p3_uqvqdiv_rvalue = IndR_ssp585_p3_uqvqdiv_regress["rvalue"]
IndR_ssp585_p3_uqvqdiv_pvalue = IndR_ssp585_p3_uqvqdiv_regress["pvalue"]

In [19]:
uqvqdivhis_ver_JJA[0,:,:,:].min()

<xarray.DataArray 'div_uqvq' ()>
array(-4.23826746e-06)
Coordinates:
    models   <U10 'ACCESS-CM2'
Attributes:
    standard_name:  eastward_wind
    long_name:      Eastward Wind
    units:          100kg/(m*s)
    comment:        Zonal wind (positive in a eastward direction).
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-09-25T21:26:57Z altered by CMOR: Reordered dimension...

In [18]:
uqvqdivERA5_ver_JJA.min()

<xarray.DataArray 'u' ()>
array(-4.12821558e-06)
Attributes:
    standard_name:  eastward_wind
    long_name:      U component of wind
    units:          100kg/(m*s)

In [ ]:
fq_his_ver_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/hus_historical_r144x72_195001-201412.nc")
q_his_ver_JJA = fq_his_ver_JJA["hus"]

In [20]:
preAIR_JJA.coords["time"] = uqERA5_ver_JJA.coords["time"]
(
    IndRAIR_ERA5_uq_slope,
    IndRAIR_ERA5_uq_intercept,
    IndRAIR_ERA5_uq_rvalue,
    IndRAIR_ERA5_uq_pvalue,
    IndRAIR_ERA5_uq_hypothesis,
) = ca.dim_linregress(preAIR_JJA, uqERA5_ver_JJA)

(
    IndRAIR_ERA5_vq_slope,
    IndRAIR_ERA5_vq_intercept,
    IndRAIR_ERA5_vq_rvalue,
    IndRAIR_ERA5_vq_pvalue,
    IndRAIR_ERA5_vq_hypothesis,
) = ca.dim_linregress(preAIR_JJA, vqERA5_ver_JJA)

(
    IndRAIR_ERA5_uqvqdiv_slope,
    IndRAIR_ERA5_uqvqdiv_intercept,
    IndRAIR_ERA5_uqvqdiv_rvalue,
    IndRAIR_ERA5_uqvqdiv_pvalue,
    IndRAIR_ERA5_uqvqdiv_hypothesis,
) = ca.dim_linregress(preAIR_JJA, uqvqdivERA5_ver_JJA)

In [22]:
#   save the regression results

lat=IndRAIR_ERA5_uq_slope.coords["lat"]
lon=IndRAIR_ERA5_uq_slope.coords["lon"]

IndRAIR_ERA5_uq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndRAIR_ERA5_uq_slope.data),
        intercept=(["lat", "lon"], IndRAIR_ERA5_uq_intercept.data),
        rvalue=(["lat", "lon"], IndRAIR_ERA5_uq_rvalue.data),
        pvalue=(["lat", "lon"], IndRAIR_ERA5_uq_pvalue.data),
        hypothesis=(["lat", "lon"], IndRAIR_ERA5_uq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uq fields of ERA5 regress onto 1979-2014 AIR"),
)


IndRAIR_ERA5_vq_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndRAIR_ERA5_vq_slope.data),
        intercept=(["lat", "lon"], IndRAIR_ERA5_vq_intercept.data),
        rvalue=(["lat", "lon"], IndRAIR_ERA5_vq_rvalue.data),
        pvalue=(["lat", "lon"], IndRAIR_ERA5_vq_pvalue.data),
        hypothesis=(["lat", "lon"], IndRAIR_ERA5_vq_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="vq fields of ERA5 regress onto 1979-2014 AIR"),
)



IndRAIR_ERA5_uqvqdiv_regress = xr.Dataset(
    data_vars=dict(
        slope=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_slope.data),
        intercept=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_intercept.data),
        rvalue=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_rvalue.data),
        pvalue=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_pvalue.data),
        hypothesis=(["lat", "lon"], IndRAIR_ERA5_uqvqdiv_hypothesis.data),
    ),
    coords=dict(
        lat=lat.data,
        lon=lon.data,
    ),
    attrs=dict(description="uqvqdiv fields of ERA5 regress onto 1979-2014 AIR"),
)



IndRAIR_ERA5_uq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_uq_regress.nc")

IndRAIR_ERA5_vq_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_vq_regress.nc")

IndRAIR_ERA5_uqvqdiv_regress.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/SASMR_reg_result/IndRAIR_ERA5_uqvqdiv_regress.nc")
